In [ ]:
!pip install pydicom
!pip install git+https://github.com/JoHof/lungmask
import numpy as np 
import os
import pydicom
import matplotlib.pyplot as plt
from lungmask import mask
import SimpleITK as sitk
from glob import glob
import h5py
from datetime import datetime 
import pytz
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from scipy import ndimage

from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 1.9MB 8.2MB/s 
  Cloning https://github.com/JoHof/lungmask to /tmp/pip-req-build-ek3wtpvm
  Running command git clone -q https://github.com/JoHof/lungmask /tmp/pip-req-build-ek3wtpvm
     |████████████████████████████████| 44.9MB 72kB/s 
     |████████████████████████████████| 1.2MB 59.5MB/s 
     |████████████████████████████████| 3.6MB 52.2MB/s 
  Created wheel for lungmask: filename=lungmask-0.2.8-cp36-none-any.whl size=17169 sha256=6312ad306d5a5cef01a1420d494df036012bfddef6206cfa231e209494d6943d
  Stored in directory: /tmp/pip-ephem-wheel-cache-65tdy7i4/wheels/59/2e/c0/1365551c1853d11aef03baf7ed94cdac831342c7f8a03e45ca
Successfully built lungmask
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
covid_path = '/content/drive/My Drive/covid/'
normal_path = '/content/drive/My Drive/normal/'
covid_masked_path = '/content/drive/My Drive/data_do_not_delete/dataset/covid/'
normal_masked_path = '/content/drive/My Drive/data_do_not_delete/dataset/normal/'
masking_log_file_start_path = '/content/drive/My Drive/data_do_not_delete/logs/mask_log/masking_'

def displaySampleStack(stack, rows=8, cols=8, start_with=0, show_every=1, cmap='gray', size=12):
  '''Plots the given np.ndarray as rows x cols stack of images'''
  fig,ax = plt.subplots(rows,cols,figsize=[size, size])
  for i in range(rows*cols):
      ind = start_with + i*show_every
      ax[int(i/rows),int(i % rows)].set_title('slice %d' % ind)
      ax[int(i/rows),int(i % rows)].imshow(stack[ind],cmap=cmap)
      ax[int(i/rows),int(i % rows)].axis('off')
  plt.show()

def getTimeStamp():
  '''Returns current time as time stamp in IST'''
  IST = pytz.timezone('Asia/Kolkata') 
  return datetime.now(IST).strftime("%d-%m-%Y-%H:%M:%S")

def logPrint(file, content):
  '''Prints to log file and console'''
  file.write("\n" + getTimeStamp() + "\t" + content)
  print(getTimeStamp(), content)

def check():
  '''Checking'''
  op = []
  for id in range(1, 100):
    curr = getPatientDCMSFolder(id)
    filepath = os.listdir(curr)[0]
    dcmfile = pydicom.dcmread(curr + '/' + filepath)
    ctscan = dcmfile[(0x0040, 0x0275)][0][(0x0040, 0x0009)].value[:9]
    op.append([id, ctscan])
  return op

def getPatientDCMSFolder(id):
  '''Returns path to the Patient's DCM folder'''
  return covid_path + 'patient-%02d/'%(id)

def getNonPatientDCMSFolder(id):
  '''Returns path to Non patient's DCM folder'''
  return normal_path + 'npatient-%02d/'%(id)

def load_scan(path):
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
    for s in slices:
        s.SliceThickness = slice_thickness
    return slices


def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)
    # Set outside-of-scan pixels to 1
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
    image += np.int16(intercept)
    return np.array(image, dtype=np.int16)


def read_dicom_file(filepath):
    """Read and load volume"""
    img = load_scan(filepath)
    scan = get_pixels_hu(img)
    return scan

def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[0]
    current_width = img.shape[1]
    current_height = img.shape[-1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height

    # Resize across z-axis
    img = ndimage.zoom(img, (depth_factor, width_factor, height_factor), order=1)
    return img

def process_scan(path):
    """Read and resize volume"""
    # Read scan
    print('About to read DICOM at', path)
    volume = read_dicom_file(path)
    print('CT Scan read successful from', path)
    print(type(volume), volume.shape)

    # Resize width, height and depth
    volume = resize_volume(volume)
    print('Resizing successful')
    print(volume.shape)
    plt.imshow(volume[30], cmap="gray")
    return volume

def applyMask(img):
  '''Returns original sampled images and segmented images after masking'''
  # The original resized image
  orig = sitk.GetImageFromArray(img, isVector=False)
  seg = mask.apply(orig)
  return img, np.multiply(img, seg)

def segmentation(patient_id_from, patient_id_to=None, covid=True):
  patient_id_to = patient_id_from if patient_id_to is None else patient_id_to
  mask_log = masking_log_file_start_path + getTimeStamp() + '.txt'
  log = open(mask_log, "a")

  for patient_id in range(patient_id_from, patient_id_to + 1):
    logPrint(log, ("" if covid else "Non ") + "Patient ID = " + str(patient_id))
    # Segmentation and mask
    path = getPatientDCMSFolder(patient_id) if covid else getNonPatientDCMSFolder(patient_id)
    img = process_scan(path)
    orig_np, seg_np = applyMask(img)
    # Save
    save_path = (covid_masked_path if covid else (normal_masked_path + "n")) + "masked_%02d.npy" % (patient_id)
    logPrint(log, "Saving file at = " + save_path)
    np.save(save_path, seg_np)
    logPrint(log, "Segmentation and saving successful for " + ("" if covid else "Non ") + "Patient " + str(patient_id) + "\n\n")

  # After Segmentation of last patient
  displaySampleStack(np.load((covid_masked_path if covid else (normal_masked_path + "n")) + "masked_%02d.npy"%(patient_id_to)))
  log.close()

In [ ]:
def get_scoring_model(width=128, height=128, depth=64, final_activation="linear"):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((depth, width, height, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation=final_activation)(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model

scoring_model_linear = get_scoring_model(width=128, height=128, depth=64, final_activation="linear")
scoring_model_linear.summary()

Model: "3dcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 128, 128, 1)] 0         
_________________________________________________________________
conv3d (Conv3D)              (None, 62, 126, 126, 64)  1792      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 31, 63, 63, 64)    0         
_________________________________________________________________
batch_normalization (BatchNo (None, 31, 63, 63, 64)    256       
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 29, 61, 61, 64)    110656    
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 14, 30, 30, 64)    0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 30, 30, 64)    256   

In [ ]:
def get_severity_class(score):
  """Returns severity for the given score"""
  if 0 <= score < 20:
    return 'Low'
  if 20 <= score < 40:
    return 'Mild'
  if 40 <= score < 60:
    return 'Moderate'
  if 60 <= score < 80:
    return 'Severe'
  if score >= 80:
    return 'Critical'

scoring_model_linear.load_weights("/content/drive/My Drive/data_do_not_delete/dataset/scoring_linear_3d.h5")


In [ ]:
def get_severity_num(score):
  """Returns severity for the given score"""
  if 0 <= score < 20:
    return 1
  if 20 <= score < 40:
    return 2
  if 40 <= score < 60:
    return 3
  if 60 <= score < 80:
    return 4
  if score >= 80:
    return 5

In [ ]:
def read_many_hdf5(h5file):
  """Loads dataset from given h5file"""
  images = np.array(h5file["images"]).astype("int32")
  labels = np.array(h5file["labels"]).astype("uint8")
  scores = np.array(h5file["scores"]).astype("uint8")
  return images, labels, scores
  
vali = h5py.File('/content/drive/My Drive/data_do_not_delete/dataset/validation.h5', 'r')
x_val, y_val, scores_val = read_many_hdf5(vali)
vali.close()

In [ ]:
pred_score = scoring_model_linear.predict(x_val, batch_size=5)

pred_class = np.reshape(np.array([get_severity_num(score) for score in pred_score]), (len(pred_score), 1))
actual_class = np.reshape(np.array([get_severity_num(score) for score in scores_val]), (len(pred_score), 1))

In [ ]:
import pandas as pd
array = np.concatenate((pred_class, actual_class), axis = 1)
data_frame = pd.DataFrame(data = array,  
                  columns = ['Predicted Severity', 'Actual Severity'])
print(data_frame) 

     Predicted Severity  Actual Severity
0                     1                1
1                     1                1
2                     1                1
3                     1                1
4                     1                1
..                  ...              ...
165                   3                3
166                   1                2
167                   1                1
168                   1                1
169                   3                3

[170 rows x 2 columns]


In [ ]:
data_frame.to_csv('/content/drive/My Drive/data_do_not_delete/dataset/predicted_severity_num_train.csv', index=False)

In [ ]:
def get_model(width=128, height=128, depth=64):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((depth, width, height, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model

In [ ]:
model = get_model(width=128, height=128, depth=64)
model.summary()

Model: "3dcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64, 128, 128, 1)] 0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 62, 126, 126, 64)  1792      
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 31, 63, 63, 64)    0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 31, 63, 63, 64)    256       
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 29, 61, 61, 64)    110656    
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 14, 30, 30, 64)    0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 14, 30, 30, 64)    256   

In [ ]:
model.load_weights("/content/drive/MyDrive/data_do_not_delete/dataset/3d_image_classification.h5")
results = model.predict(x_val, batch_size=5)

In [ ]:
y_pred = [[1] if r > 0.5 else [0] for r in results]

In [ ]:
y_pred

[[1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1]]

In [ ]:
np.column_stack((y_pred, y_val))

array([[1, 1],
       [0, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 1],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [0, 1],
       [1, 1],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 0],
       [0, 0],
       [1, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1]])

In [ ]:
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
confusion_matrix(y_val, y_pred)

array([[12,  4],
       [ 5, 19]])

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()

In [ ]:
tn

12

In [ ]:
tp

19

In [ ]:
fp

4

In [ ]:
fn

5

In [ ]:
precision_recall_fscore_support(y_val, y_pred, beta=1.0)

(array([0.70588235, 0.82608696]),
 array([0.75      , 0.79166667]),
 array([0.72727273, 0.80851064]),
 array([16, 24]))

In [ ]:
precision = tp / (tp + fp)
precision

0.8260869565217391

In [ ]:
recall = tp / (tp + fn)
recall

0.7916666666666666

In [ ]:
f1 = 2 * precision * recall / (precision + recall)
f1

0.8085106382978724